### Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора 
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук `parse_rospotrebnadzor.ipynb`
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать 
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [1]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 

--2022-10-18 16:41:18--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.18
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: /s/raw/fnpq3z4bcnoktiv/positive.csv [переход]
--2022-10-18 16:41:20--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc277755833a331fe1716980b0a1.dl.dropboxusercontent.com/cd/0/inline/BvAd_b9KfFVi4LIOBdrs30OxT-OhgPdlIcOeLJyWfuUciePoM3in2uUYB7q83653-ULq7HMaQVNJ3UVf3gFob7xSMVZoC60WNAWXawmmZ6B0CAmTROy5Cs13rYB0kkgaOEpCm66a_xT5Iu9YgbmG2waN7F8feW64niToSoarE24ODw/file# [переход]
--2022-10-18 16:41:20--  https://uc277755833a331fe1716980b0a1.dl.dropboxusercontent.com/cd/0/inline/BvAd_b9KfFVi4LIOBdrs30OxT-OhgPdlIcOeLJyWfuUciePoM3in2uUYB7q83653-ULq7HMaQVNJ3UVf3gFob7xSMVZoC60W

In [2]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-10-18 16:41:23--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.18
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: /s/raw/r6u59ljhhjdg6j0/negative.csv [переход]
--2022-10-18 16:41:25--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc8ae7afd503ffa0d76ca0937381.dl.dropboxusercontent.com/cd/0/inline/BvCHzXwN-1jyjjSXog914M7hoQDSndTbhC3nlv87WdxFI0BhncogfpkZUdue5vf0-G81V3PoHcqJJsKpineooHtS9qyGSGK_4PT4OW05MwPQlcrG2lekef-EB2x0jH9UOeWUQ6Dga60QqeJjE2T-l2peGmOL8s84mUKkG4CkMTlfkQ/file# [переход]
--2022-10-18 16:41:25--  https://uc8ae7afd503ffa0d76ca0937381.dl.dropboxusercontent.com/cd/0/inline/BvCHzXwN-1jyjjSXog914M7hoQDSndTbhC3nlv87WdxFI0BhncogfpkZUdue5vf0-G81V3PoHcqJJsKpineooHtS9qyGSGK_

In [3]:
!pip install pyLDAvis

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [4]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
import nltk

import pymorphy2
from gensim.models import *
from gensim import corpora

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [5]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [6]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = pd.concat([positive, negative], ignore_index=True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [7]:
words_regex = re.compile(r'\w+')

def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4]

morph = pymorphy2.MorphAnalyzer() 
stopwords_list = stopwords.words('russian')

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] #список нормальных форм
    return [w for w in lemmas if not w in stopwords and w.isalpha()] #чистим список токенов

def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    ignore_list = [r'[A-za-z]',r'\w+ще\b',r'который',r'\bидио\w+',r'почему',
                   r'\bпид\w+',r'весь',r'значит',r'\bхуй\w+',r'\bхуй\w+',r'\w+xу.\b',
                   r'\bбл\w+',r'\w+xу\w+',r'\w+но\b',r'очень',r'\bпиз\w+',r'это',
                   r'\bгов\w+',r'\bпи.де\w+',r'лох',r'нету',r'ой',r'мочь',r'\w+ть\b',
                   r'\bфиг\w+',r'сука',r'\bпоп\w+',r'свой',r'капец']
    for i in ignore_list:
        string_tokens = re.sub(i, ' ',  string_tokens)

    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

/usr/local/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.9/site-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

In [8]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, немного, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция, хотя]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


In [9]:
dictionary = corpora.Dictionary(df['text'])
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')
len(dictionary.keys())

8184

In [10]:
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [11]:
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=5, chunksize=50, update_every=1, passes=2)

In [12]:
lda.show_topics(num_topics=5, num_words=10, formatted=True)

[(0,
  '0.022*"ночь" + 0.018*"любимый" + 0.015*"остаться" + 0.014*"фильм" + 0.014*"хотя" + 0.013*"казаться" + 0.012*"город" + 0.012*"твит" + 0.010*"пара" + 0.010*"лето"'),
 (1,
  '0.029*"жизнь" + 0.021*"утро" + 0.019*"голова" + 0.016*"наш" + 0.015*"выходн" + 0.014*"нравиться" + 0.013*"вроде" + 0.012*"минута" + 0.012*"рука" + 0.012*"ранний"'),
 (2,
  '0.026*"время" + 0.023*"хотеться" + 0.019*"друг" + 0.018*"последний" + 0.017*"работа" + 0.016*"первый" + 0.016*"пока" + 0.016*"новый" + 0.015*"вчера" + 0.015*"прийти"'),
 (3,
  '0.058*"день" + 0.053*"завтра" + 0.045*"просто" + 0.030*"школа" + 0.023*"идти" + 0.020*"урок" + 0.020*"мама" + 0.018*"час" + 0.018*"неделя" + 0.018*"каждый"'),
 (4,
  '0.059*"сегодня" + 0.038*"человек" + 0.034*"дом" + 0.031*"больш" + 0.027*"хороший" + 0.023*"самый" + 0.022*"жаль" + 0.019*"никто" + 0.012*"жалко" + 0.012*"дело"')]

In [13]:
print(lda.log_perplexity(corpus))
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

-8.52313228191057
Персплексия:  0.00019881487630181555


In [14]:
visual_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(visual_data)

/usr/local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is de

In [15]:
pyLDAvis.save_html(visual_data, 'hw04.html')

Получившиеся тематики:\
1.Повседневность\
2.Дружеские отношения\
3.Учеба\
4.Времяпрепровождение\
5.Досуг

Нужно заметить, что результаты нестабильны в силу самого алгоритма, плюс для реального использования необходим значительный объем работ по фильтрации и отбрасыванию слов для более четкого разграничения по темам